## 0. First time user? Run this first to download the relevant functionality for the program 

### Otherwise, *skip*  

In [1]:
%pip install selenium
%pip install parsel
%pip install time
%pip install pandas
%pip install bs4
%pip install urllib
%pip install request
%pip install numpy
%pip install openpyxl

## 1. Run this to import relevant functionality for the program

In [15]:
import selenium
import parsel
import time
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from parsel import Selector
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen
from openpyxl import load_workbook
import numpy as np

## 2. Download Chrome Driver

Link:
https://chromedriver.chromium.org/downloads
- make sure you download the chromedriver that is compatible with your google chrome version
- check you google chrome version by clicking three dots on top right of the Google Chrome browser > Help > About Google Chrome

## 3. Open Chrome Driver & LinkedIn 

Set 'path' as the file pathway to where Chrome Driver is stored on your device

In [3]:
path = 'G:\Sales-Shared\- CPA\LinkedIn Prospecting\chromedriver_win32/chromedriver.exe'
driver = webdriver.Chrome(path)

In [4]:
# Will take the user to LinkedIn
# Please login
driver.get('https://www.linkedin.com/sales')

### Enter your LinkedIn credentials and then enter a search query on Sales Navigator

Once you've entered your search query and LinkedIn Sales Navigator has generated a list of leads on your screen, you can proceed to the code below

## 4. Familiarise yourself with the following functions

### Need to know functions

**"Functions" begin with "def" (in green in the code blocks below). These 2 functions in particular require user input and understanding:**
- **get_page_links_all(num_pages)**
    - purpose: returns a list of user profile links on a specified number of search result pages; runs chronologically
    - **requried user input:** number of search result pages, 'num_pages', you want to obtain user profile links from
        - if no input is specified, the function fails; the function doesn't default to collecting links on all of the pages in the search query if nothing is specified
    - example: if you wanted to scrape LinkedIn data on 14 pages, you would type in 14 between the parentheses like this: get_page_links_all(14)
- **get_all_data(links,file_name)**
    - purpose: scrapes profile data from all the links fed into it and saves that data into an excel file
        - profile data includes: name, location, company, role, a bio, and the user's LinkedIn url
        - links is the list of linkes you collected by using the "get_page_links_all()" function
        - 'file_name' is the name of the excel file you want the data saved into
    - **required user input:** list of links, 'links', and the name of the excel file, 'file_name' you want to store your data into
    - example: get_all_data(proper_links,'prospect file 1.xlsx')

**You can ignore all other functions. But if you're curious, you can continue reading the documentation below:**

In [8]:
def get_page_links():
    """
    Returns a list of the unique user profile links from a single page of a search query on Sales Navigator. 
    
    The function will scroll down the search page collecting links because the links on
    Sales Navigator do not load until they are shown on screen. 
    """
    
    sel = Selector(text=driver.page_source)
    good_links = []
    
    y = 0
    for timer in range(0,12):
        sleep(1)
        driver.execute_script("window.scrollTo(0, "+str(y)+")")
        y += 500 
    
    all_links = driver.find_elements_by_tag_name('a')
    for link in all_links:
        if link.get_attribute('href').startswith('https://www.linkedin.com/sales/people/'):
            good_links.append(link.get_attribute('href'))
    return (list(set(good_links))) 

In [9]:
def get_page_links_all(num_pages):
    """
    Returns a list of unique user profile links from a specified number of pages of a search query on Sales Navigator. 
    
    The function collect links over a certain number of pages. Once done with the current 
    page, the function will click the "Next" button to move on to the next page and 
    scrape those links (and so on).
    
    If you do not specify a the number of pages you want to scrape data from, the function
    will return an error. If you specify a number of pages in excess of the number of 
    search result pages generated, then the function will also return an error.
    """
    links = []
    for i in range(0,num_pages):
        sleep(0.2)
        
        sel = Selector(text=driver.page_source)
        
        y = 200
        for timer in range(0,12):
            sleep(0.2)
            driver.execute_script("window.scrollTo(0, "+str(y)+")")
            y += 500  
             
        for link in driver.find_elements_by_tag_name('a'):
            if link.get_attribute('href').startswith('https://www.linkedin.com/sales/people/'):
                links.append(link.get_attribute('href'))

        sleep(1)
        button = driver.find_element_by_class_name('search-results__pagination-next-button')
        
        if button.click() != None:
            button.click()
                     
    return list(set(links))

In [10]:
def get_single_data_point():
    
    """
    Obtains a single user's name, role & firm, location, and self-bio from their Sales Navigator user profile page.
    
    Appends that information to the following lists:
    names = [] (stores the user's name)
    roles_and_firms = [] (stores the user's role & firm)
    locations = [] (stores the user's location)
    abouts = [] (stores the user's self-bio)
    
    Please initialise these lists before using this function.
    """
    
    sleep(1)
    # get names
    potential_name_block = driver.find_elements_by_tag_name('span')
    for name in potential_name_block:
        if name.get_attribute('class').startswith('profile-topcard-person-entity__name'):
            names.append(name.text)
    
    sleep(0.5)
    # get titles
    potential_title_block = driver.find_elements_by_tag_name('dd')
    for title in potential_title_block:
        if title.get_attribute('class').startswith('mt2'):
            roles_and_firms.append(title.text)
    
    sleep(0.5)
    # get locations
    potential_location_block = driver.find_elements_by_tag_name('div')
    for location in potential_location_block:
        if location.get_attribute('class').startswith('profile-topcard__location'):
            locations.append(location.text)
            
    sleep(0.5)
    # get more info
    # click 'see more' button first
    potential_see_more_button = driver.find_elements_by_tag_name('button')
    for button in potential_see_more_button:
        if button.get_attribute('class').startswith('button--unstyled link-without-visited-state inline-block'):
            if button.click() != None:
                sleep(1)
                button.click()
    
    potential_about_block = driver.find_elements_by_tag_name('div')
    for about in potential_about_block:
        if about.get_attribute('class').startswith('profile-topcard__summary-modal-content'):
            if len(about.text) > 0:
                abouts.append(about.text)
            else:
                potential_about_block = driver.find_elements_by_tag_name('span')
                for about in potential_about_block:
                    if about.get_attribute('class').startswith('display-block overflow-hidden profile-topcard__summary-content'):
                        abouts.append(about.text)

In [11]:
def get_all_data(links,file_name):
    
    """
    Stores multiple users' names, roles & firms, locations, self-bios, and urls into a 
    user specified Excel file
    
    This function accepts a list of links which you can collect 
    via the get_page_links_all(num_pages) function and an existing Excel file's name.
    
    The function will also show the user a progress counter for the number of results 
    that have been collected.
    
    Example: 
    proper_links = get_page_links_all(12)
    get_all_data(proper_links,'prospect list 1.xlsx')
    """
    
    workbook = load_workbook(file_name)
    worksheet = workbook.worksheets[0]
    
    counter = 1
    for i in range(0,len(links)):
        driver.get(links[i])
        
        sleep(1)
        get_single_data_point()
        
        if len(names) > 0:
            worksheet[f'b{i+2}'] = (names.pop())
        else:
            worksheet[f'b{i+2}'] = (None)
            
        if len(roles_and_firms) > 0:
            worksheet[f'c{i+2}'] = (roles_and_firms.pop())
        else:
            worksheet[f'c{i+2}'] = (None)
            
        if len(locations) > 0:
            worksheet[f'd{i+2}'] = (locations.pop())
        else: 
            worksheet[f'd{i+2}'] = (None)
        
        if len(abouts) > 0:
            worksheet[f'e{i+2}'] = (abouts.pop())
        else:
            worksheet[f'e{i+2}'] = (None)
        
        worksheet[f'f{i+2}'] = (driver.current_url)
        
        print(f"Progress: {counter} / {len(links)}")
        counter += 1
        
        workbook.save(file_name)

## 5. Web Scraping on LinkedIn 

***CAUTION:*** remember to have your search query results up before running any of the code below!

## Workflow

1. Collect the links of prospects' Sales Navigator profiles
2. Save these links to an Excel file for future reference
3. Collect user profile data and save that to an Excel file 

## 1. Get all relevant links 

In [86]:
# Create empty list to store links
proper_links = []

In [87]:
# Get x pages of links
proper_links = get_page_links_all(12)

In [88]:
# Check how many links we scraped from all pages; there should be 25 links per full page
len(proper_links)

300

### 1.1 I'd recommend saving the list of links you've collected to an Excel file

- With the following blocks of code below, you can save the list of links that you've collected to an Excel file 
- This is useful because when you close this program (i.e. close this tab in Google Chrome), the list of links stored in the "proper_links" variable will need to be collected again
- By saving your links, you won't need to scrape the data again; instead, you can just load it from the excel file you saved your links in
- make the file name informative like: "Prospects from Google.xlsx"

In [89]:
links_df = pd.DataFrame(proper_links,columns = ['urls'])

How code block below works:
1. **links_df.to_excel("links.xlsx",sheet_name='URLs', columns=links_df.columns)**
    - creates a new Excel file with the name "links.xlsx" 
    - YOU CAN CHANGE the name of "links.xlsx" to create a NEW Excel file, with an informative name
    - if you already have a file called "links.xlsx", running this block of code may override your original file or produce an error message 
    - ALWAYS KEEP ".xlsx" in the name, otherwise you'll get an error

In [90]:
links_df.to_excel("Biotech Prospect Links.xlsx",sheet_name='URLs', columns=links_df.columns)

### 1.2 Load up links from an Excel file you've saved 

In [61]:
load_links = pd.read_excel("Biotech Prospect Links.xlsx")

In [62]:
proper_links = list(load_links['urls'])

## 2. Scrape data from Sales Navigator 

How the code block below works:
1. **final_df = pd.DataFrame(columns = ['names','role & firm','location','about','urls'])**
    - creates a new dataframe in Python -- you **CAN IGNORE** 
2. **final_df.to_excel("output_digital_transformation.xlsx",sheet_name='Prospect Info', columns=final_df.columns)**
    - creates a new Excel file with the name "Biotech Prospects" 
    - note: *you can change* "Biotech Prospects.xlsx" to create a new Excel file with a different name
    - if you already have a file called "Biotech Prospects.xlsx", running this block of code may override your original file OR produce an error
    - ALWAYS KEEP ".xlsx" in the name, otherwise you'll get an error

In [91]:
final_df = pd.DataFrame(columns = ['names','role & firm','location','about','urls'])
final_df.to_excel("Biotech Prospects.xlsx",sheet_name='Prospect Info', columns=final_df.columns)

In [13]:
# RUN THIS to instatiate the lists necessary to collect our data from LinkedIn
# data to extract
names = []
roles_and_firms = []
locations = []
abouts = []

In [12]:
get_all_data((proper_links),"Biotech Prospects.xlsx")

## IMPORTANT EXTRA NOTE

Sometimes get_all_data((proper_links),"file_name.xlsx") outputs an **error** while executing, but do not fret: get_all_data((proper_links),"file_name.xlsx") **autosaves** information in Excel while executing, so **you will retain all data scraped up to the point of failure.**

Example: if get_all_data((proper_links),"file_name.xlsx") breaks at 195/300; I will still have 195 data points on user profiles

To get the remaining data, you can: 
- create a new excel file (with a different name) via the following code:
    - final_df = pd.DataFrame(columns = ['names','role & firm','location','about','urls'])
    - final_df.to_excel("file_name_2.xlsx",sheet_name='Prospect Info', columns=final_df.columns)
- then run this final line of code to collect the rest of the info:
    - get_all_data((proper_links[196:300]),"file_name_2.xlsx")

# Further Exploration

If your team's sales people cover different regions, roles, or firms, you can develop functionality on top of this tool to group and organise prospects by these attributes. This will improve your team's prospecting process because it will:
1. Help you analyse the number of prospects per region/firm/role and therefore give the team a better idea of how to split the prospects across different team members
2. Increase your team's efficiency because it will help each person find their respective list of targets more efficienctly

### Sample Exploration

In [16]:
canada_prospects = pd.read_excel('Sustainability Prospects in Canada.xlsx')

In [19]:
canada_prospects = canada_prospects.drop('Unnamed: 0',axis=1)

In [21]:
canada_prospects.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 650 entries, 0 to 649
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   names        650 non-null    object
 1   role & firm  650 non-null    object
 2   location     650 non-null    object
 3   about        472 non-null    object
 4   urls         650 non-null    object
dtypes: object(5)
memory usage: 25.5+ KB


In [28]:
# notice that firm name usually comes after the key word "at"
# so we can split our column role & firm using that information
canada_prospects.head()

,names,role & firm,location,about,urls
0,Gilbert Sabat,Digital Evangelist and Organization Evolution ...,"Oakville, Ontario, Canada",I have 21 years of experience in the transform...,https://www.linkedin.com/sales/people/ACwAAADC...
1,Patrick Li,Vice President Of Engineering at KioSoft Techn...,"Greater Toronto Area, Canada",Patrick's current focus is on coaching and tra...,https://www.linkedin.com/sales/people/ACwAAACT...
2,"Siraj Berhan, MBA, Certified Enterprise Coach","Director, Agile Enablement | Championing for a...","Toronto, Ontario, Canada",Over 17 years of progressive software developm...,https://www.linkedin.com/sales/people/ACwAAAAT...
3,Ed Norrena,Pres and CEO at EDGE Environmental Consulting LTD,"Ottawa, Ontario, Canada",Ed Norrena is a civil engineer/executive with ...,https://www.linkedin.com/sales/people/ACwAAAO4...
4,Natasha Walji,Director @ Google | McKinsey | Yale | Cambridg...,"Toronto, Ontario, Canada","Director at Google leading the Telecom, Tech &...",https://www.linkedin.com/sales/people/ACwAAAAG...


In [26]:
# example of split
canada_prospects['role & firm'].apply(lambda x: x.split('at'))[0]

['Digital Evangelist and Organiz', 'ion Evolution Advisor']